In [ ]:
!pip install scipy

import pandas as pd
import numpy as np
from scipy.stats import f_oneway

# from google.colab import drive
# drive.mount('/content/drive')
# cd drive/MyDrive/Gaze_Detection_System/RT_Raw_Data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
  def filter_RTs(data):
    # data = datasets[0]
    data.head(5)
    data = data.rename(columns={"RespMat_ 1": "Index", "RespMat_ 2": "Trial number", "RespMat_ 3": "Block number", "RespMat_ 4": "Block trial number", "RespMat_ 5": "Arrow direction", "RespMat_ 6": "Scrambled?"\
                                , "RespMat_ 7": "Correct response", "RespMat_ 8": "Response", "RespMat_ 9": "Correct?", "RespMat_10": "RT"\
                                , "RespMat_11": "T/D same/diff", "RespMat_12": "D emotion", "RespMat_13": "which T"})


    data = data.tail(-1)
    data.head(3)
    og = data.shape[0]

    # Remove incorrect trials
    data = data[data['Correct?'] == '1']
    print(data.shape)

    # Remove RT's slower than 200ms
    data['RT'] = data['RT'].astype(float)
    data['D emotion'] = data['D emotion'].astype(float) # Also do for 
    data = data[data['RT'] < 2.0]


    # # Remove 3 standard deviations from the mean
    data = data[np.abs(data.RT-data.RT.mean()) <= (3*data.RT.std())]


    print(f"Removed: {og - data.shape[0]} datapoints")

    return data

In [ ]:
def get_ANOVA_dataframe(data, emotions):
  # Get t
  # emotions = ['Happy', 'Fearful', 'Neutral']
  # anova_dataset = pd.DataFrame({'Happy': [], 'Fearful': [], 'Neutral':[]})

  mean_emo_RT_list = []
  for emo_num in range(1,4): # For every distractor emotion

    non_scram_data = data[data['Scrambled?']=='1']
    emo_series = non_scram_data[non_scram_data['D emotion']==emo_num] # Reduce the dataframe to only contain that emotion
    mean_emo_RT_list.append(np.mean(emo_series['RT']))


  
  scram_data = data[data['Scrambled?']=='2']
  mean_emo_RT_list.append(scram_data['RT'].dropna().mean())


  return mean_emo_RT_list


def do_ANOVA(anova_dataset):
  fvalue, pvalue = f_oneway(anova_dataset['Happy'], anova_dataset['Fearful'], anova_dataset['Neutral'], anova_dataset['Scrambled'])
  print(fvalue, pvalue)

In [ ]:
# 2 means scrambled

def get_raw_data(env):
  datasets = []
  for i in range(1,8):
    datasets.append(pd.read_csv(f'pp{i}_{env}.csv'))
  return datasets

def setup_anova_dataset(datasets):
  RTs = []
  scram_mean_RTs = []
  FD_mean_RTs_nonscram = []

  anova_dataset = pd.DataFrame({'Happy': [], 'Fearful': [], 'Neutral':[], 'Scrambled':[]})
  for i in range(len(datasets)):
    data = datasets[i]
    data = filter_RTs(data)

    anova_dataset.loc[len(anova_dataset)] = get_ANOVA_dataframe(data, anova_dataset.columns)
  return anova_dataset


def get_ANOVA_results(env):

  datasets = get_raw_data(env)

  anova_dataset = setup_anova_dataset(datasets)
  
  do_ANOVA(anova_dataset)

# Normally distributed!
def check_normality(env):

  datasets = get_raw_data(env)
  anova_dataset = setup_anova_dataset(datasets)
  for col in anova_dataset.columns:
    print(stats.shapiro(anova_dataset[col]))



Run Code

In [ ]:
from scipy import stats
# Check for normality
check_normality('N')

print('---------------------------------')
print('---------------------------------')
print('---------------------------------')

get_ANOVA_results('N')


(525, 13)
Removed: 61 datapoints
(469, 13)
Removed: 116 datapoints
(473, 13)
Removed: 103 datapoints
(517, 13)
Removed: 70 datapoints
(474, 36)
Removed: 102 datapoints
(523, 13)
Removed: 63 datapoints
(513, 13)
Removed: 69 datapoints
(0.9701437950134277, 0.8994835615158081)
(0.9802538752555847, 0.9607608914375305)
(0.9713175296783447, 0.9077767729759216)
(0.9859598278999329, 0.9832692742347717)
---------------------------------
---------------------------------
---------------------------------
(525, 13)
Removed: 61 datapoints
(469, 13)
Removed: 116 datapoints
(473, 13)
Removed: 103 datapoints
(517, 13)
Removed: 70 datapoints
(474, 36)
Removed: 102 datapoints
(523, 13)
Removed: 63 datapoints
(513, 13)
Removed: 69 datapoints
0.028693562945296566 0.9932730637991614


In [ ]:
# Create table of means and stdevs



(525, 13)
Removed: 61 datapoints
(469, 13)
Removed: 116 datapoints
(473, 13)
Removed: 103 datapoints
(517, 13)
Removed: 70 datapoints
(474, 36)
Removed: 102 datapoints
(523, 13)
Removed: 63 datapoints
(513, 13)
Removed: 69 datapoints


,Happy,Fearful,Neutral,Scrambled
0,0.554058,0.545000,0.513212,0.537724
1,0.551883,0.585162,0.579660,0.561798
2,0.655111,0.686745,0.691767,0.643395
3,0.484374,0.476003,0.471156,0.451394
4,0.768140,0.744521,0.782365,0.734297
5,0.514225,0.526425,0.525101,0.525008
6,0.359300,0.368335,0.362897,0.352043


In [ ]:
datasets = get_raw_data('C')
anova_dataset = setup_anova_dataset(datasets)
anova_dataset

mean_list = list(anova_dataset.mean())
stdev_list = list(anova_dataset.std()) 

pd.DataFrame(data = [mean_list,stdev_list] , columns=anova_dataset.columns, index = ['mean', 'stdev'])


(518, 13)
Removed: 68 datapoints
(460, 13)
Removed: 121 datapoints
(526, 13)
Removed: 61 datapoints
(507, 13)
Removed: 76 datapoints
(477, 13)
Removed: 99 datapoints
(558, 13)
Removed: 29 datapoints
(413, 13)
Removed: 172 datapoints


,Happy,Fearful,Neutral,Scrambled
mean,0.550543,0.555119,0.55694,0.520739
stdev,0.124094,0.128698,0.13032,0.117562
